# Requirements

In [1]:
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
import numpy as np
import scipy as sp
import scipy.integrate
import scipy.optimize
import sympy
output_notebook()

Loading BokehJS ...

# No drag

The equations of motion of a projectile fired by a cannon is given by:
$$
  \frac{d^2 \vec{r}}{d t^2} = -g \vec{e}_y
$$
The value for $g$ is $9.81 m/s^2$, and the initial velocity $v_0$ is $700 m/s$.

In [7]:
g = 9.81

## Equations

As usual, the second order differential equation is transformed into a set of first order equations, and the right hand side of the equations is defined by the function `rhs`.  Additionally, the Jacobian is provided.

In [2]:
def rhs(t, Y, g):
    x, y, v_x, v_y = Y
    v = np.sqrt(v_x**2 + v_y**2)
    return np.array([
        v_x,
        v_y,
        0.0,
        -g,
    ])

In [3]:
def jac(t, Y, g):
    x, y, v_x, v_y = Y
    v = np.sqrt(v_x**2 + v_y**2)
    return np.array([
        [0.0, 0.0, 1.0, 0.0],
        [0.0, 0.0, 0.0, 1.0],
        [0.0, 0.0, 0.0, 0.0],
        [0.0, 0.0, 0.0, 0.0],
    ])


Since it is more convenient to approach the problem using degrees rather than radians, a function is defined to compute the initial values.

In [4]:
def compute_init_values(alpha, v_0 = 700):
    alpha_rad = np.pi*alpha/180.0
    return np.array([0.0, 0.0,
                     v_0*np.cos(alpha_rad), v_0*np.sin(alpha_rad),
                     ])

## Solution

We can solve the set of equations for an angle of 40 degrees.

In [5]:
init_values = compute_init_values(40.0)
x_0, y_0, v_x_0, v_y_0 = init_values

Define the ODE system, setting initial conditions and parameters.

In [8]:
sys = scipy.integrate.ode(rhs, jac) \
           .set_integrator('dopri5') \
           .set_initial_value(init_values, 0.0) \
           .set_f_params(g) \
           .set_jac_params(g)

In [8]:
delta_t = 0.5e-2

In [9]:
t, x, y, v_x, v_y = [0.0], [x_0], [y_0], [v_x_0], [v_y_0]

In [10]:
while sys.successful() and y[-1] >= 0:
    sys.integrate(sys.t + delta_t)
    t.append(sys.t)
    x.append(sys.y[0])
    y.append(sys.y[1])
    v_x.append(sys.y[2])
    v_y.append(sys.y[3])

In [11]:
fig = figure(plot_width=500, plot_height=300,
             x_axis_label='x', y_axis_label='y')
fig.line(x, y)
show(fig)

## Range as a function of the firing angle

In [9]:
def compute_range(alpha, delta_t = 1.0e-3,
                  rhs=rhs, jac=jac,
                  rhs_params=(g, ), jac_params=(g, )):
    init_values = compute_init_values(alpha)
    x_0, y_0, v_x_0, v_y_0 = init_values
    sys = scipy.integrate.ode(rhs, jac) \
               .set_integrator('dopri5') \
               .set_initial_value(init_values, 0.0) \
               .set_f_params(*rhs_params) \
               .set_jac_params(*jac_params)
    t, x, y, v_x, v_y = [0.0], [x_0], [y_0], [v_x_0], [v_y_0]
    while sys.successful() and y[-1] >= 0:
        sys.integrate(sys.t + delta_t)
        t.append(sys.t)
        x.append(sys.y[0])
        y.append(sys.y[1])
        v_x.append(sys.y[2])
        v_y.append(sys.y[3])
    return 0.5*(x[-2] + x[-1])

In [10]:
alphas = np.linspace(10.0, 80.0, 50)
ranges = [compute_range(alpha) for alpha in alphas]

In [11]:
fig = figure(plot_width=500, plot_height=300,
             x_axis_label='alpha', y_axis_label='range')
fig.line(alphas, ranges)
show(fig)

## Maximal range

In [12]:
def helper_func(alpha):
    return -compute_range(alpha)

In [13]:
scipy.optimize.minimize_scalar(helper_func, bracket=(10.0, 80.0),
                               method='golden')

     fun: -49949.26057595005
    nfev: 44
     nit: 39
 success: True
       x: 44.97543056302328

# Drag

The equations of motion of a projectile fired by a cannon is given by:
$$
  \frac{d^2 \vec{r}}{d t^2} = -g \vec{e}_y - \frac{B_2}{m} v e^{-\frac{y}{y_d}} \vec{v} 
$$
Realistic values for $g = 9.81 \frac{m}{s^2}$, $\frac{B_2}{m} = 4\cdot10^{-5} m^{-1}$, $y_d = 10^4 m$.

In [14]:
def rhs(t, Y, g, b_2_m, y_d):
    x, y, v_x, v_y = Y
    v = np.sqrt(v_x**2 + v_y**2)
    return [
        v_x,
        v_y,
        -b_2_m*v*np.exp(-y/y_d)*v_x,
        -g - b_2_m*v*np.exp(-y/y_d)*v_y,
    ]

## Jacobian

Computing the Jacobian is not really hard, but a little tedious.  It can easily be done using sympy.

In [15]:
b_2_m, v, y, y_d, v_x, v_y = sympy.symbols('b_2_m v y y_d v_x v_y')

In [16]:
v = sympy.sqrt(v_x**2 + v_y**2)

In [17]:
rhs_v_x = -b_2_m*v*sympy.exp(-y/y_d)*v_x

In [18]:
sympy.diff(rhs_v_x, y)

b_2_m*v_x*sqrt(v_x**2 + v_y**2)*exp(-y/y_d)/y_d

In [19]:
sympy.diff(rhs_v_x, v_x)

-b_2_m*v_x**2*exp(-y/y_d)/sqrt(v_x**2 + v_y**2) - b_2_m*sqrt(v_x**2 + v_y**2)*exp(-y/y_d)

In [20]:
sympy.diff(rhs_v_x, v_y)

-b_2_m*v_x*v_y*exp(-y/y_d)/sqrt(v_x**2 + v_y**2)

In [21]:
def jac(t, Y, g, b_2_m, y_d):
    x, y, v_x, v_y = Y
    v = np.sqrt(v_x**2 + v_y**2)
    return np.array([
        [0.0, 0.0, 1.0, 0.0],
        [0.0, 0.0, 0.0, 1.0],
        [0.0, -b_2_m*v_x*v*np.exp(-y/y_d)/y_d, -b_2_m*v_x**2*np.exp(-y/y_d)/v - b_2_m*v*np.exp(-y/y_d), -b_2_m*v_x*v_y*np.exp(-y/y_d)/v],
        [0.0, -b_2_m*v_y*v*np.exp(-y/y_d)/y_d, -b_2_m*v_x*v_y*np.exp(-y/y_d)/v, -b_2_m*v_t**2*np.exp(-y/y_d)/v - b_2_m*v*np.exp(-y/y_d)],
    ])

## Solution

In [22]:
b_2_m, y_d = 4.0e-5, 1.0e4

In [23]:
init_values = compute_init_values(40.0)
x_0, y_0, v_x_0, v_y_0 = init_values

Define the ODE system, setting initial conditions and parameters.

In [24]:
sys = scipy.integrate.ode(rhs, jac) \
           .set_integrator('dopri5') \
           .set_initial_value(init_values, 0.0) \
           .set_f_params(g, b_2_m, y_d) \
           .set_jac_params(g, b_2_m, y_d)

In [25]:
delta_t = 0.5e-2

In [26]:
t, x, y, v_x, v_y = [0.0], [x_0], [y_0], [v_x_0], [v_y_0]

In [27]:
while sys.successful() and y[-1] >= 0:
    sys.integrate(sys.t + delta_t)
    t.append(sys.t)
    x.append(sys.y[0])
    y.append(sys.y[1])
    v_x.append(sys.y[2])
    v_y.append(sys.y[3])

In [40]:
fig = figure(plot_width=500, plot_height=300,
             x_axis_label='x', y_axis_label='y')
fig.line(x, y)
show(fig)

## Total energy

It is useful to check that the total energy decreases monotonically.  This is the sum of the kinetic and the potential energy, so
$$
    \frac{E}{m} = \frac{v_x^2 + v_y^2}{2} + g y
$$

In [41]:
def total_energy(x, y, v_x, v_t, g):
    return 0.5*(np.array(v_x)**2 + np.array(v_y)**2) + g*np.array(y)

In [42]:
energy = total_energy(x, y, v_x, v_y, g)

In [43]:
fig = figure(plot_width=500, plot_height=300,
             x_axis_label='t', y_axis_label='E')
fig.line(t, energy)
show(fig)

## Maximal range

Again, we can compute the maximal range of the artillery piece by maximizing the following helper function.

In [29]:
def helper_func(alpha):
    return -compute_range(alpha, rhs=rhs, jac=jac,
                          rhs_params=(g, b_2_m, y_d),
                          jac_params=(g, b_2_m, y_d))

In [30]:
scipy.optimize.minimize_scalar(helper_func, bracket=(10.0, 80.0),
                               method='golden')

     fun: -26602.28108246449
    nfev: 44
     nit: 39
 success: True
       x: 45.94188834356312

Taking drag into account, the firing angle to get the maximal range increases by almost a degree, and the range is almost halved.